In [2]:
import numpy as np
import os
from pathlib import Path
import cv2

import datetime as dt

import tensorflow as tf
import utils
from rsl_depth_completion.data.components.raw_data_loaders import depth_read, img_read
from kbnet import eval_utils
import yaml
import pandas as pd
%matplotlib inline

2023-04-20 10:25:37.847813: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-20 10:25:37.921983: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-20 10:25:38.296760: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/master/.conda/envs/ssdc/lib/python3.10/site-packages/cv2/../../lib64:
2023-04-20 10:25:38.29

In [3]:
common_config = yaml.safe_load(open("common.yaml"))
split = common_config["split"]
config = {**common_config}


In [4]:
path_to_input_img = Path(config["path_to_input_img"])
path_to_sparse_dm = Path(config["path_to_sparse_dm"])
path_to_gt = Path(config["path_to_gt"])

mask_result_dir = config["mask_dir"]
masked_img_names = [
    x[x.rfind("/") + 1 :]
    for x in open(config["image_filelist_path"]).read().splitlines()
]
mask_result_dir = Path(mask_result_dir)
# all masks, but inference was interrupted and not all outputs are available
assert len(masked_img_names) >= len(os.listdir(path_to_input_img))


In [5]:
ts = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
img2class_logdir = f"tb_logs/masked_predictions/{split}/img2class_" + ts
class2img_logdir = f"tb_logs/masked_predictions/{split}/class2img_" + ts
# !rm -rf tb_logs
img2class_file_writer = tf.summary.create_file_writer(img2class_logdir)
class2img_file_writer = tf.summary.create_file_writer(class2img_logdir)


2023-04-20 10:25:41.293219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-20 10:25:41.309050: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-20 10:25:41.309207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-20 10:25:41.309776: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

In [9]:
from collections import defaultdict

model_names = ["kbnet", "penet", "nlspn"]

errors_per_class = defaultdict(
    lambda: {
        "metrics": {
            "rmse": {k: 0.0 for k in model_names},
            "mae": {k: 0.0 for k in model_names},
        },
        "count": 0,
        "imgs_with_no_gt": [],
        "imgs_with_no_prediction": {k: [] for k in model_names},
    }
)
errors_per_img = defaultdict(
    lambda: {
        "metrics": {
            "rmse": {k: 0.0 for k in model_names},
            "mae": {k: 0.0 for k in model_names},
        },
        "num_objects": 0,
        "objects": [],
        "masks_with_no_gt": [],
        "masks_with_no_prediction": {k: [] for k in model_names},
    }
)
fig_classes = defaultdict(list)
n_first_imgs_to_consider=300
input_img_names = sorted(os.listdir(path_to_input_img))[:n_first_imgs_to_consider]


In [15]:
objects_to_consider = [
    # smooth surfaces
    "road",
    "wall",
    "terrain",
    # rough surfaces
    "vegetation",
    # objects with sharp edges & small objects
    "pole",
    "person",
    "rider",
    # objects with holes
    "bicycle",
]
pixels_per_class = defaultdict(int)


In [16]:
from tqdm import tqdm


# n_imgs_to_evaluate = 0
n_imgs_to_evaluate = 30

# predicted_dm_scaler = 1000.0
predicted_dm_scaler = 1.0

img_idxs = np.random.choice(len(input_img_names), n_imgs_to_evaluate, replace=False)

with img2class_file_writer.as_default():
    for counter, img_idx in tqdm(enumerate(img_idxs), total=n_imgs_to_evaluate):
        img_name = input_img_names[img_idx]
        img_name = img_name.replace(".png", "")
        img_name_in_index_format = f"{img_idx:010d}"

        masked_img_name_idx = masked_img_names.index(f"{img_name}.png")
        if masked_img_name_idx == -1:
            print(f"Skipping {img_name} as it has no segmentation masks.")
            continue
        masked_img_name = masked_img_names[masked_img_name_idx]

        sparse_dm = depth_read(
            path_to_sparse_dm
            / f"{img_name}.png".replace("_image_", "_velodyne_raw_", 1)
        ).squeeze()
        gt_dm = depth_read(
            path_to_gt / f"{img_name}.png".replace("_image_", "_groundtruth_depth_", 1)
        ).squeeze()
        img = img_read(path_to_input_img / f"{img_name}.png")

        pred_dms = []
        are_all_preds_loaded = True
        for model_name in model_names:
            model_config = yaml.safe_load(open(f"{model_name}_val.yaml"))
            path_to_result_dir = Path(model_config["result_dir"])
            path_to_pred_dm = Path(path_to_result_dir)

            try:
                pred_dm = depth_read(
                    path_to_pred_dm / f"{img_name_in_index_format}.png"
                ).squeeze()
            except OSError as e:
                print(f"{model_name} prediction for {img_name} is unavailable due to:\n{e}")
                errors_per_class[model_name]["imgs_with_no_prediction"].append(img_name)
                errors_per_img[model_name]["masks_with_no_prediction"].append(
                    masked_img_name
                )
                are_all_preds_loaded = False
                break
            pred_dms.append(pred_dm)
            
        if not are_all_preds_loaded:
            print(f"Skipping {img_name} as not all predictions are available.")
            continue

        num_objects = 0
        figs_masked = []
        figs_full = []

        fig = utils.plot_full_results_for_all_models(
            model_names,
            pred_dms,
            img,
            title=f"{img_name}",
            save_path="./test_full.png",
        )
        fig_full = utils.plot_to_image(fig)
        figs_full.append(fig_full)

        for obj_class_with_ext in tqdm(
            os.listdir(mask_result_dir / masked_img_name), leave=False
        ):
            obj_class = obj_class_with_ext.split(".")[0]
            if obj_class not in objects_to_consider:
                continue
            binary_mask = (
                cv2.imread(
                    str(mask_result_dir / masked_img_name / obj_class_with_ext),
                    cv2.IMREAD_GRAYSCALE,
                )
                / 255
            )
            gt_dm_masked = gt_dm * binary_mask
            if len(np.nonzero(gt_dm_masked)[0]) == 0:
                print(f"No GT depth values for the class: {obj_class}")
                errors_per_class[obj_class]["imgs_with_no_gt"].append(img_name)
                errors_per_img[img_name]["masks_with_no_gt"].append(obj_class)
                continue

            for model_name, pred_dm in zip(model_names, pred_dms):
                pred_dm_masked = pred_dm * binary_mask

                mae = eval_utils.mean_abs_err(
                    predicted_dm_scaler * pred_dm_masked, predicted_dm_scaler * gt_dm_masked
                )
                rmse = eval_utils.root_mean_sq_err(
                    predicted_dm_scaler * pred_dm_masked, predicted_dm_scaler * gt_dm_masked
                )

                if len(np.nonzero(pred_dm_masked)[0]) == 0:
                    errors_per_img[img_name]["masks_with_no_prediction"][
                        model_name
                    ].append(obj_class)
                    errors_per_class[obj_class]["imgs_with_no_prediction"][
                        model_name
                    ].append(img_name)

                errors_per_class[obj_class]["metrics"]["rmse"][
                    model_name
                ] += rmse
                errors_per_class[obj_class]["metrics"]["mae"][model_name] += mae

                errors_per_img[img_name]["metrics"]["rmse"][model_name] += rmse
                errors_per_img[img_name]["metrics"]["mae"][model_name] += mae

            num_pixels = len(np.nonzero(binary_mask)[0])
            pixels_per_class[obj_class] += num_pixels

            errors_per_class[obj_class]["count"] += 1
            errors_per_img[img_name]["num_objects"] += 1
            errors_per_img[img_name]["objects"].append(obj_class)

            fig = utils.plot_masked_results_for_all_models(
                model_names,
                pred_dms,
                img,
                binary_mask,
                obj_class,
                title=f"{obj_class}-{img_name}",
                save_path="./test_masked.png",
            )
            fig_masked = utils.plot_to_image(fig)

            figs_masked.append(fig_masked)
            fig_classes[obj_class].append(fig_masked)
            # break

        utils.log_imgs(f"{img_name}/masked", figs_masked)
        utils.log_imgs(f"{img_name}/full", figs_full)
        tb_name_prefix = f"{img_name}"
        utils.log_errors_per_img(errors_per_img, img_name, tb_name_prefix, model_names)

        if counter >= n_imgs_to_evaluate:
            break
        # break


 10%|█         | 3/30 [00:11<01:50,  4.10s/it]

No GT depth values for the class: rider


 27%|██▋       | 8/30 [00:29<01:22,  3.77s/it]

No GT depth values for the class: person
No GT depth values for the class: rider


 43%|████▎     | 13/30 [00:47<01:01,  3.63s/it]

...

 73%|███████▎  | 22/30 [01:20<00:30,  3.83s/it]

No GT depth values for the class: person


 87%|████████▋ | 26/30 [01:34<00:14,  3.72s/it]

No GT depth values for the class: person


100%|██████████| 30/30 [01:47<00:00,  3.57s/it]


In [31]:
with class2img_file_writer.as_default():
    for obj_class, figs in fig_classes.items():
        utils.log_imgs(f"{obj_class}/imgs", figs)
        utils.log_errors_per_class(errors_per_class, obj_class, model_names)


Note about the results:

- mean pixel error is in micrometers
- for some images, no ground truth available

In [20]:
model_mae_per_category = {}
model_mae_per_category_unsorted = {}
for model_name in model_names:
    model_errors = {}
    for cat, cat_stats in errors_per_class.items():
        model_errors[cat] = round((
            cat_stats["metrics"]["mae"][model_name] / pixels_per_class[cat]
        ) * 1e6, 3)
    model_mae_per_category[model_name] = sorted(
        model_errors.items(), key=lambda x: x[1], reverse=True
    )
    model_mae_per_category_unsorted[model_name] = model_errors.items()
pd.DataFrame(model_mae_per_category_unsorted)


,kbnet,penet,nlspn
0,"(terrain, 35.789)","(terrain, 36.022)","(terrain, 35.946)"
1,"(wall, 56.912)","(wall, 56.996)","(wall, 57.024)"
2,"(pole, 70.545)","(pole, 61.905)","(pole, 61.264)"
3,"(road, 30.291)","(road, 30.377)","(road, 30.386)"
4,"(bicycle, 22.956)","(bicycle, 23.047)","(bicycle, 22.96)"
5,"(rider, 27.442)","(rider, 27.399)","(rider, 27.34)"
6,"(vegetation, 118.554)","(vegetation, 82.059)","(vegetation, 89.168)"
7,"(person, 55.708)","(person, 56.232)","(person, 56.067)"


In [21]:
pd.DataFrame(model_mae_per_category).to_csv("model_mae_per_category.csv")


In [37]:
for writer in [img2class_file_writer, class2img_file_writer]:
    with writer.as_default():
        for model_name in model_names:
            model_config = yaml.safe_load(open(f"{model_name}_val.yaml"))
            path_to_result_dir = model_config["result_dir"]
            tf.summary.text(
                f"meta/{model_name}/result_dir",
                str(path_to_result_dir),
                step=0,
            )